Installations

In [ ]:
!pip install transformers

In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

Loading summarisation pipeline

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Getting the blog post

In [16]:
URL = "https://medium.com/ai-in-plain-english/%EF%B8%8F-langchain-streamlit-llama-bringing-conversational-ai-to-your-local-machine-a1736252b172"

In [17]:
r = requests.get(URL)

In [18]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [19]:
ARTICLE

"Sign up Sign In Sign up Sign In 🦜️ LangChain + Streamlit🔥+ Llama\xa0🦙: Bringing Conversational AI to Your Local Machine 🤯 Afaque Umer Follow Artificial Intelligence in Plain English -- 31 Listen Share In the past few months, Large Language Models (LLMs) have gained significant attention, capturing the interest of developers across the planet. These models have created exciting prospects, especially for developers working on chatbots, personal assistants, and content creation. The possibilities that LLMs bring to the table have sparked a wave of enthusiasm in the Developer | AI | NLP community. What are LLMs? Large Language Models (LLMs) refer to machine learning models capable of producing text that closely resembles human language and comprehending prompts in a natural manner. These models undergo training using extensive datasets comprising books, articles, websites, and other sources. By analyzing statistical patterns within the data, LLMs predict the most probable words or phrases

Chunking the text

In [12]:
max_chunk = 500

In [20]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [21]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [22]:
len(chunks)

7

Summarising text

In [23]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [27]:
res

[{'summary_text': ' Large Language Models (LLMs) are machine learning models capable of producing text that closely resembles human language and comprehending prompts in a natural manner . By utilizing LLMs, we can incorporate domain-specific data to address inquiries effectively . In this article, I will demonstrate the process of creating your own Document Assistant from the ground up, utilizing LLaMA 7b and Langchain .'},
 {'summary_text': ' LLaMA is a new large language model designed by Meta AI, which is Facebook’s parent company . With a diverse collection of models ranging from 7 billion to 65 billion parameters, it stands out as one of the most comprehensive language models available . To execute the LLM on a local CPU, we need a local model in GGML format .'},
 {'summary_text': ' We will use llama-cpp-python which is a Python binding for llama. cpp which acts as an Inference of the LLaMA model in pure C/C++ . The primary interface through which you can interact with them is th

In [25]:
text = ' '.join([summ['summary_text'] for summ in res])
text

' Large Language Models (LLMs) are machine learning models capable of producing text that closely resembles human language and comprehending prompts in a natural manner . By utilizing LLMs, we can incorporate domain-specific data to address inquiries effectively . In this article, I will demonstrate the process of creating your own Document Assistant from the ground up, utilizing LLaMA 7b and Langchain .  LLaMA is a new large language model designed by Meta AI, which is Facebook’s parent company . With a diverse collection of models ranging from 7 billion to 65 billion parameters, it stands out as one of the most comprehensive language models available . To execute the LLM on a local CPU, we need a local model in GGML format .  We will use llama-cpp-python which is a Python binding for llama. cpp which acts as an Inference of the LLaMA model in pure C/C++ . The primary interface through which you can interact with them is through text . We have successfully executed our first LLM on th